<a href="https://colab.research.google.com/github/foroughkarandish/Convolutional-Neural-Network-Architecture/blob/master/Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project is not started yet and I am thinking about the best way to explain and implement it. Stay tuned for updates!!!

In this project I will explain CNN and use them to build an image classifier.

1. [Introduction](#1)<br>
2. [Load Packages](#2)<br>
    A. [Import](#21)<br>
    B. [Setup](#22)<br>
    C. [Version](#23)<br>
3. [Problem Definition](#3)<br>
    A. [Aim](#31)<br>
    B. [Variables](#32)<br>
4. [CNN Architecture](#4)<br>
  A. [Computer vision](#41)<br>
  B. [Edge detection example](#42)<br>
  C. [Padding](#43)<br>
  D. [Strided convolution](#44) <br>
  E. [Convolutions over volumes](#45) <br>
  F. [One Layer of a Convolutional Network](#46) <br>
  G. [A simple convolution network example](#47) <br>
  H. [Pooling layers](#48) <br>
  I. [Convolutional neural network example](#49) <br> 
  J. [Why convolutions?](#499) <br>
5. [Model developement](#5)<br>
    A. [Keras](#51)<br>   
6. [References](#6)<br>

1. introduction

This course will teach you how to build convolutional neural networks and apply it to image data. 